In [1]:
# Step 1: Setup (imports and reproducible randomness)
import random
random.seed(42)  # deterministic for demonstration; remove or change seed in production

# Use a large modulus to keep shares non-negative and avoid wrap-around issues
MOD = 10**9 + 7

# Step 2: Define private values (secrets) for each participant
# (In an actual private run, you would NOT print these values;
#  we print here only for demonstration/verification.)
secrets = {
    "Ahmed": 100,
    "Ali": 60,
    "Rashida": 60
}

print("Private secrets (for demo only):", secrets)
print("Expected true total:", sum(secrets.values()))
print()

# Step 3: Each participant splits their secret into 3 random shares (one per participant)
def make_shares(secret, n_parties=3, mod=MOD):
    # generate n-1 random shares, last share ensures sum modulo mod equals secret
    shares = [random.randint(0, mod-1) for _ in range(n_parties - 1)]
    last = (secret - sum(shares)) % mod
    shares.append(last)
    return shares

# create shares_per_party: dict participant -> list of shares (index corresponds to receiver id)
participants = list(secrets.keys())
shares_per_party = {}

for p in participants:
    shares_per_party[p] = make_shares(secrets[p], n_parties=len(participants), mod=MOD)

# Step 4: Distribute shares: build received_shares[receiver] = list of shares received from all parties
received_shares = {p: [] for p in participants}
for i, sender in enumerate(participants):
    sender_shares = shares_per_party[sender]
    for j, receiver in enumerate(participants):
        # sender_shares[j] is intended for receiver
        received_shares[receiver].append(sender_shares[j])

# Print shares (for demo/verification)
print("Shares created by each participant (rows = sender, columns = shares for [Ahmed, Ali, Rashida]):")
for p in participants:
    print(f"{p:8}: {shares_per_party[p]}")
print()

print("Shares received by each participant (each list contains one share from every sender):")
for p in participants:
    print(f"{p:8}: {received_shares[p]}")
print()

# Step 5: Each participant computes their local sum = sum of shares they received (mod MOD)
local_sums = {}
for p in participants:
    local_sums[p] = sum(received_shares[p]) % MOD

print("Local sums computed by each participant:")
for p in participants:
    print(f"{p:8}: {local_sums[p]}")
print()

# Step 6: Combine local sums to get the global total (mod MOD)
global_sum = sum(local_sums.values()) % MOD
print("Global sum reconstructed from local sums:", global_sum)

# Verification vs true sum
true_sum = sum(secrets.values()) % MOD
print("True sum (for verification):", true_sum)
print("Match? ->", global_sum == true_sum)


Private secrets (for demo only): {'Ahmed': 100, 'Ali': 60, 'Rashida': 60}
Expected true total: 220

Shares created by each participant (rows = sender, columns = shares for [Ahmed, Ali, Rashida]):
Ahmed   : [686579303, 119540831, 193879973]
Ali     : [26855092, 796233790, 176911185]
Rashida : [295310485, 262950628, 441738954]

Shares received by each participant (each list contains one share from every sender):
Ahmed   : [686579303, 26855092, 295310485]
Ali     : [119540831, 796233790, 262950628]
Rashida : [193879973, 176911185, 441738954]

Local sums computed by each participant:
Ahmed   : 8744873
Ali     : 178725242
Rashida : 812530112

Global sum reconstructed from local sums: 220
True sum (for verification): 220
Match? -> True


# Part I: MPC Sum using Secret Sharing

## Description
Three parties — Ahmed, Ali, and Rashida — each hold private data (e.g., salaries).  
They want to compute the total sum without revealing their individual values.  
This demonstrates **Secure Multi-Party Computation (SMPC)** using **Secret Sharing**.

## Steps
1. Each participant splits their private number into random shares.
2. Each participant distributes shares to all others.
3. Each participant sums the received shares (including their own share).
4. The global sum is computed by combining all participants' summed shares.

## Expected Output
The correct total (e.g., 220) is calculated **without revealing any individual's private number**.

## Observations
- Privacy is maintained: no participant learns another's private value.  
- The sum is correctly computed, demonstrating that SMPC allows secure collaborative computation.


In [2]:
import random

MOD = 10**9 + 7  # Prime for modular arithmetic

# Generate polynomial coefficients (degree = k-1)
def generate_polynomial(secret, k):
    coeffs = [secret]
    for _ in range(k - 1):
        coeffs.append(random.randint(1, MOD-1))
    return coeffs

# Evaluate polynomial at x
def evaluate_polynomial(coeffs, x):
    value = 0
    for power, c in enumerate(coeffs):
        value = (value + c * pow(x, power, MOD)) % MOD
    return value

# Create n shares
def generate_shares(secret, k, n):
    coeffs = generate_polynomial(secret, k)
    shares = []
    for x in range(1, n+1):
        shares.append((x, evaluate_polynomial(coeffs, x)))
    return shares

# Reconstruct secret using Lagrange interpolation
def lagrange_interpolation(shares, mod=MOD):
    secret = 0
    k = len(shares)
    for i in range(k):
        xi, yi = shares[i]
        li = 1
        for j in range(k):
            if i != j:
                xj, _ = shares[j]
                li *= (0 - xj) * pow(xi - xj, mod-2, mod)
                li %= mod
        secret = (secret + yi * li) % mod
    return secret

# Example usage
secret = 123
k = 3
n = 5

shares = generate_shares(secret, k, n)
print("Shares:", shares)

chosen = random.sample(shares, k)
print("\nChosen Shares:", chosen)

reconstructed = lagrange_interpolation(chosen)
print("\nReconstructed Secret:", reconstructed)
print("Match:", reconstructed == secret)


Shares: [(1, 389498542), (2, 78652368), (3, 67461608), (4, 355926262), (5, 944046330)]

Chosen Shares: [(1, 389498542), (5, 944046330), (3, 67461608)]

Reconstructed Secret: 123
Match: True


# Part II: Shamir’s Secret Sharing Implementation

## Description
This part demonstrates **Shamir's Secret Sharing (SSS)**, a threshold scheme where a secret can be split into multiple shares.  
Any **k out of n shares** can reconstruct the secret, while fewer than k shares reveal nothing.  
The scheme uses **polynomial-based secret sharing** and **modular arithmetic**.

## Steps in the Code
1. **Define a prime modulus (`MOD`)** for arithmetic to avoid overflow.  
2. **Generate a polynomial** of degree `k-1` with the secret as the constant term.  
3. **Evaluate the polynomial** at `n` different points to create `n` shares.  
4. **Reconstruct the secret** from any `k` chosen shares using Lagrange interpolation.  
5. **Test the scheme** by checking that the reconstructed secret matches the original.

## Example Execution
- Secret = 123  
- Threshold `k` = 3, total shares `n` = 5  
- Output shows the generated shares, selected shares for reconstruction, and the recovered secret.  

## Observation
- The scheme allows the secret to be recovered **only when enough shares are combined**.  
- Fewer than `k` shares reveal **no information** about the secret.


In [9]:
import os
import random
from cryptography.fernet import Fernet

# Generate two labels (keys) for bit 0 and bit 1
def gen_label():
    return Fernet.generate_key()

# Encrypt output label using two input labels
def double_encrypt(a_label, b_label, out_label):
    f1 = Fernet(a_label)
    f2 = Fernet(b_label)
    return f1.encrypt(f2.encrypt(out_label))

# Decrypt using two labels
def double_decrypt(a_label, b_label, ciphertext):
    try:
        f1 = Fernet(a_label)
        f2 = Fernet(b_label)
        return f2.decrypt(f1.decrypt(ciphertext))
    except:
        return None


# Step 1: Generate labels for A and B

A0, A1 = gen_label(), gen_label()
B0, B1 = gen_label(), gen_label()

# Labels for output 0 and 1
OUT0, OUT1 = gen_label(), gen_label()

# Step 2: Build the garbled AND table
garbled_table = []

garbled_table.append(double_encrypt(A0, B0, OUT0))  # 0 AND 0
garbled_table.append(double_encrypt(A0, B1, OUT0))  # 0 AND 1
garbled_table.append(double_encrypt(A1, B0, OUT0))  # 1 AND 0
garbled_table.append(double_encrypt(A1, B1, OUT1))  # 1 AND 1

random.shuffle(garbled_table)

# Step 3: Omar and Nancy inputs
A = 1  # Omar input bit
B = 0  # Nancy input bit

omar_label = A1 if A == 1 else A0
nancy_label = B1 if B == 1 else B0

# Step 4: Evaluate garbled circuit
result_label = None
for item in garbled_table:
    out = double_decrypt(omar_label, nancy_label, item)
    if out is not None:
        result_label = out
        break

#
# Step 5: Interpret result
output_bit = 1 if result_label == OUT1 else 0

print("Omar Input (A):", A)
print("Nancy Input (B):", B)
print("A AND B =", output_bit)


Omar Input (A): 1
Nancy Input (B): 0
A AND B = 0


# Part III: Garbled Circuits (AND Gate)

## Description
This part demonstrates a **garbled circuit implementation of an AND gate**.  
Two participants, Omar and Nancy, provide private input bits (A and B).  
The AND gate output is computed **without revealing their individual inputs**, using encrypted labels.

## Steps in the Code
1. **Generate labels (keys)** for input bits 0 and 1 for both Omar (A) and Nancy (B).  
2. **Generate labels for output bits** 0 and 1.  
3. **Build the garbled table**: encrypt each possible output with the corresponding input labels.  
4. **Shuffle the table** to hide the order of entries.  
5. **Convert participant inputs to labels**.  
6. **Evaluate the garbled table** using the participant labels to obtain the encrypted output.  
7. **Interpret the result** by comparing the decrypted label with output labels to get the actual bit.

## Example Execution
- Omar input (A) = 1  
- Nancy input (B) = 0  
- The code prints the AND gate output: `A AND B = 0`.

## Observation
- Privacy is preserved: Omar and Nancy’s input bits are never revealed directly.  
- The garbled circuit correctly computes the AND result using only encrypted labels.
